In [1]:
#!pip install langchain-community==0.2.4 langchain==0.2.3 faiss-cpu==1.8.0 unstructured==0.14.5 unstructured[pdf]==0.14.5 transformers==4.41.2 sentence-transformers==3.0.1

**Importing the Dependencies**

In [5]:
import os

from langchain_community.llms import Ollama
from langchain.document_loaders import UnstructuredFileLoader
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQA

In [6]:
# loading the LLM
llm = Ollama(
    model="llama3:instruct",
    temperature=0
)

In [7]:
# loading the document
loader = UnstructuredFileLoader("NIPS-2017-attention-is-all-you-need-Paper.pdf")
documents = loader.load()

In [9]:
# create document chunks
text_splitter = CharacterTextSplitter(separator="/n",
                                      chunk_size=1000,
                                      chunk_overlap=200)

In [11]:
text_chunks = text_splitter.split_documents(documents)

In [12]:
# loading the vector embedding model
embeddings = HuggingFaceEmbeddings()

/home/ubuntu/myenv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/home/ubuntu/myenv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/home/ubuntu/myenv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new dow

In [15]:
knowledge_base = FAISS.from_documents(text_chunks, embeddings)

In [16]:
# retrieval QA chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=knowledge_base.as_retriever())

In [17]:
question = "What is this document about?"
response = qa_chain.invoke({"query": question})
print(response["result"])

This document appears to be a research paper or article about the Transformer, a sequence transduction model based entirely on attention, which outperforms previous architectures in translation tasks. The authors discuss their approach, its advantages, and future directions for applying attention-based models to other tasks.


In [18]:
question = "What is the architecture discussed in the model?"
response = qa_chain.invoke({"query": question})
print(response["result"])

The architecture discussed in the model is the Transformer, which is a sequence transduction model based entirely on attention, replacing recurrent layers commonly used in encoder-decoder architectures with multi-headed self-attention.
